<a href="https://colab.research.google.com/github/gyuholee94/NLP/blob/master/%EC%BD%9C%EA%B8%88%EB%A6%AC%EC%84%A4%EB%AA%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import tqdm
import requests

no_page = 693
dates =[]
call_rates = []
index = ['date', 'callrate']

for page in range(no_page):

  url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn'
  query = {
      'marketindexCd' : 'IRR_CALL',
      'page' : page+1
  }
  resp = requests.get(url, query)
  soup = BeautifulSoup(resp.content, 'lxml')

  date = soup.select('td.date')
  call_rate = soup.select('td.num')
  for i in range(len(date)):
    dates.append(date[i].text.strip())
    call_rates.append(call_rate[i*3-3].text.strip())
Call_rates = pd.DataFrame([dates, call_rates], index=index)
Call_rates = Call_rates.T
Call_rates.to_csv('Call_rates.csv')
#naver callrate crawl, transpose, save

In [ ]:
Call_rates

,date,callrate
0,2020.07.23,0.47
1,2020.07.22,0.46
2,2020.07.21,0.45
3,2020.07.20,0.46
4,2020.07.17,0.44
...,...,...
4844,2001.01.09,5.28
4845,2001.01.08,5.30
4846,2001.01.05,5.32
4847,2001.01.04,5.34


In [ ]:
import pandas

dt_index = pandas.date_range(start='20050401', end='20171231')

dt_list = dt_index.strftime("%Y.%m.%d").tolist()
datels = []
for i in dt_list:
    datels.append(i)
datels
#to make 'None' cells, use pandas

['2005.04.01',
 '2005.04.02',
 '2005.04.03',
 '2005.04.04',
 '2005.04.05',
 '2005.04.06',
 '2005.04.07',
 '2005.04.08',
 '2005.04.09',
 '2005.04.10',
 '2005.04.11',
 '2005.04.12',
 '2005.04.13',
 '2005.04.14',
 '2005.04.15',
 '2005.04.16',
 '2005.04.17',
 '2005.04.18',
 '2005.04.19',
 '2005.04.20',
 '2005.04.21',
 '2005.04.22',
 '2005.04.23',
 '2005.04.24',
 '2005.04.25',
 '2005.04.26',
 '2005.04.27',
 '2005.04.28',
 '2005.04.29',
 '2005.04.30',
 '2005.05.01',
 '2005.05.02',
 '2005.05.03',
 '2005.05.04',
 '2005.05.05',
 '2005.05.06',
 '2005.05.07',
 '2005.05.08',
 '2005.05.09',
 '2005.05.10',
 '2005.05.11',
 '2005.05.12',
 '2005.05.13',
 '2005.05.14',
 '2005.05.15',
 '2005.05.16',
 '2005.05.17',
 '2005.05.18',
 '2005.05.19',
 '2005.05.20',
 '2005.05.21',
 '2005.05.22',
 '2005.05.23',
 '2005.05.24',
 '2005.05.25',
 '2005.05.26',
 '2005.05.27',
 '2005.05.28',
 '2005.05.29',
 '2005.05.30',
 '2005.05.31',
 '2005.06.01',
 '2005.06.02',
 '2005.06.03',
 '2005.06.04',
 '2005.06.05',
 '2005.06.

In [ ]:
datels = datels[::-1]
dfno = pd.DataFrame([datels], index=['date'])
dfno = dfno.T
dfno

,date
0,2017.12.31
1,2017.12.30
2,2017.12.29
3,2017.12.28
4,2017.12.27
...,...
4653,2005.04.05
4654,2005.04.04
4655,2005.04.03
4656,2005.04.02


In [ ]:
k = pd.merge(dfno, Call_rates, how='left')
k
#what's funny is that, by putting 'dfno' at the left and using it as a KEY, it's unnecessary to consider timeshift of naver call rate...

,date,callrate
0,2017.12.31,NaN
1,2017.12.30,NaN
2,2017.12.29,1.52
3,2017.12.28,1.58
4,2017.12.27,1.55
...,...,...
4653,2005.04.05,NaN
4654,2005.04.04,3.23
4655,2005.04.03,NaN
4656,2005.04.02,NaN


In [ ]:
Call = k.fillna(method='bfill') # filling None cells with value of cell before them.. if the last cell's value was None, it would have becomemore complicated job.

In [ ]:
Call

,date,callrate
0,2017.12.31,1.52
1,2017.12.30,1.52
2,2017.12.29,1.52
3,2017.12.28,1.58
4,2017.12.27,1.55
...,...,...
4653,2005.04.05,3.23
4654,2005.04.04,3.23
4655,2005.04.03,3.24
4656,2005.04.02,3.24


In [ ]:
Call_A = Call.loc[:][:][:-30]
Call_B = Call.loc[:][:][30:].rename({'date':'30일전 date', 'callrate':'30일전 callrate'}, axis='columns').reset_index(drop=True)

#30 days difference = one month

In [ ]:
len(Call_A), len(Call_B)

(4628, 4628)

In [ ]:
CALLRATE = pd.merge(Call_A, Call_B, left_index=True, right_index=True)
# CALLRATE['한달전 date'][0], CALLRATE['한달전 callrate'][0] = CALLRATE['한달전 date'][1], CALLRATE['한달전 callrate'][1]# 12/31일이 그냥 슬라이싱 한거로 가져오면 12/01을 가져오게 되기 때문에 마지막 날만 맞춰줌... 근데 숫자만 31일 전으로 돌리는것 같아서 뭔가 잘못된것같네...다시해야겠다

CALLRATE#한달전을 30일전으로 따지기로 했음. 이유- 이미 많은 주말들을 fillna로 채워줬다. 소연이가 코드 준다면 타임델타 사용도 생각해볼듯.

,date,callrate,30일전 date,30일전 callrate
0,2017.12.31,1.52,2017.12.01,1.48
1,2017.12.30,1.52,2017.11.30,1.24
2,2017.12.29,1.52,2017.11.29,1.53
3,2017.12.28,1.58,2017.11.28,1.26
4,2017.12.27,1.55,2017.11.27,1.26
...,...,...,...,...
4623,2005.05.05,3.27,2005.04.05,3.23
4624,2005.05.04,3.27,2005.04.04,3.23
4625,2005.05.03,3.21,2005.04.03,3.24
4626,2005.05.02,3.30,2005.04.02,3.24


In [ ]:
# from datetime import date

# day = date(2015, 3, 28)
# time = day - pd.DateOffset(months=1)
# print(time.date())
# 한달전 구하는 방법, 적용이 힘들어서 안쓰기로 함

In [ ]:
CALLRATE['callrate']=CALLRATE['callrate'].astype(float)
CALLRATE['30일전 callrate']=CALLRATE['한달전 callrate'].astype(float)

KeyError: ignored